Demonstrates how to make a required Django field optional (not required) with using `InferExcept`.

In [ ]:
# Set up the Django environment for the example
from examples.django_setup import setup_django_environment

setup_django_environment()

Assuming the following Django model with the `id` field as required:

In [ ]:
from typing import final

from django.db import models


@final
class SomeModel(models.Model):
    id = models.AutoField(primary_key=True, null=False) # Required field here
    name = models.CharField(max_length=100)

    class Meta:
        app_label = "examples"


Assuming the following Pydantic model:

In [ ]:
from django2pydantic import BaseSchema, Infer, InferExcept, SchemaConfig


class SomeSchema(BaseSchema[SomeModel]):

    config: SchemaConfig[SomeModel] = SchemaConfig(
        model=SomeModel,
        fields= {
            "id": InferExcept(default=None), # Lets make the id field optional
            "name": Infer,
        }
    )

The following schema will be produced:

In [ ]:
from rich import print_json

print_json(data=SomeSchema.model_json_schema())

# Assert id is not required in the schema
assert "id" not in SomeSchema.model_json_schema().get("required", [])

Lets now validate with some test data without the optional field to see it works:

In [ ]:
# Some test data without the `id` field:
test_data = {
    # no `id` here
    "name": "foo"
}

# Now let's validate the data
SomeModel = SomeSchema.model_validate(test_data)

assert SomeModel
assert SomeModel.name == test_data["name"]  # type: ignore[reportUnknownMemberType]
assert SomeModel.id is None # type: ignore[reportUnknownMemberType]

print("Passed validation.")
